In [6]:
%%time
import pandas as pd
import numpy as np
import requests
import lxml.html as LH
import re                               # For regex
import datetime                         # For days to expiry calculation

import json                             # For volatility
from bs4 import BeautifulSoup

from math import sqrt, exp, log, erf    # For Black Scholes

from decimal import *
getcontext().prec = 5


###########################################
####   Assemble Black and Scholes variables
###########################################

# Expiry Dates List
# exp_dates = pd.Series(['30NOV2017', '28DEC2017', '25JAN2018'], name="Expiry")
exp_dates = ['30NOV2017', '28DEC2017', '25JAN2018']

# Create Symbol Lot Margin (symlotmargin)
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
paisa = pd.read_html(paisaurl)[1]          # It's the second table in the url

# Make first row as header
new_header = paisa.iloc[0]
paisa = paisa[1:]
paisa.columns = new_header

symlotmargin = paisa.loc[:, ["Symbol", "Mlot", "TotMgnPerLt"]]

# Combining symlotmargin and expiry dates
slm_date = pd.concat([symlotmargin] * len(exp_dates), keys=exp_dates, ignore_index=True)
slm_date.insert(0, 'Expiry', np.repeat(exp_dates, len(symlotmargin)))

# Interest rate (10-year bond yield)
inturl = requests.get("https://countryeconomy.com/bonds/india").content
intrate = float(LH.fromstring(inturl).find_class('numero')[0].text)/100

# Script name
scrip = 'PFC'
option = 'OPTSTK'
exp_date = '28DEC2017'

# NSE URL
url = "https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?&instrument="


# URL for options table        
url = url + option + "&symbol=" + scrip + "&date="+ exp_date 

# Options table
html = requests.get(url).content
nsetable = pd.read_html(html, attrs = {'id':'octable'}, header=1)[-1][:-1].drop(['Chart', 'Chart.1'], 1)

# Current underlying price
tree = LH.fromstring(html)
node = tree.xpath("//b")[0]
undprice = float(re.findall('\d+\.\d+', node.text)[0])
undprice

# Days to expiry
expiry = (datetime.datetime.strptime(exp_date, "%d%b%Y").date() - datetime.datetime.now().date()).days

# Volatility
future = 'FUTSTK'

vol_url = "https://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuoteFO.jsp?instrument=" + \
         future + "&underlying=" + scrip

url = "https://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuoteFO.jsp?underlying=PFC&instrument=FUTSTK"
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')
data=soup.find(id="responseDiv").text.strip()
d1 = json.loads(data)
sigma=float(d1['data'][0]['annualisedVolatility'])/100


# Dividend rate
divurl = "https://finance.google.com/finance?q=NSE:"+scrip
page = requests.get(divurl)
root = LH.fromstring(page.content)
dividend = float(root.findall('.//table')[2].text_content().strip().split("\n")[2].split('/')[0])/100

df = {"undprice": undprice, "strike": 100.0, "time": expiry, "rate": intrate, "sigma": sigma, "divrate": dividend}

# Black & Scholes function
def calc_bs(undprice, strike, time, rate, sigma, divrate):

    #statistics
    sigTsquared = sqrt(Decimal(time)/365)*sigma
    edivT = exp((-divrate*time)/365)
    ert = exp((-rate*time)/365)
    d1 = (log(undprice*edivT/strike)+(rate+.5*(sigma**2))*time/365)/sigTsquared
    d2 = d1-sigTsquared
    Nd1 = (1+erf(d1/sqrt(2)))/2
    Nd2 = (1+erf(d2/sqrt(2)))/2
    iNd1 = (1+erf(-d1/sqrt(2)))/2
    iNd2 = (1+erf(-d2/sqrt(2)))/2

    #outputs
    callPrice = round(undprice*edivT*Nd1-strike*ert*Nd2, 2)
    putPrice = round(strike*ert*iNd2-undprice*edivT*iNd1, 2)
    
    return callPrice, putPrice


Wall time: 8.24 s


In [7]:
slm_date.loc[:, ['Symbol', 'Expiry']]

,Symbol,Expiry
0,RCOM,30NOV2017
1,INDIAVIX,30NOV2017
2,INDIAVIX,30NOV2017
3,JPASSOCIAT,30NOV2017
4,PNB,30NOV2017
5,FORTIS,30NOV2017
6,BIOCON,30NOV2017
7,TV18BRDCST,30NOV2017
8,UNIONBANK,30NOV2017
9,BANKINDIA,30NOV2017


In [67]:
list(df1)

['Expiry', 'Symbol', 'Mlot', 'TotMgnPerLt']

In [ ]:
### Useful commands
# calc_bs(**df)
# symlotmargin[symlotmargin['Symbol'] == 'PFC']